Train summarization model

In [1]:
%pip install datasets

%pip install py7zr # For samsum dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

# Look at Dataset

Using SAMsum dataset (collection of dialogues with summaries)

In [2]:
from datasets import load_dataset

samsum_dataset = load_dataset("samsum") # Probably shouldn't just do trust remote code

# Look at dialogue
print("Dialogue: ")
print(samsum_dataset["test"][0]["dialogue"])

# Look at summary
print("\nSummary: ")
print(samsum_dataset["test"][0]["summary"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dialogue: 
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Summary: 
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


# Get PEGASUS model

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/pegasus-cnn_dailymail"

pegagus_tokenizer = AutoTokenizer.from_pretrained(model_name)

pegasus_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

# Tokenize Dataset

In [4]:
def tokenize_dataset(batch):
  input_encodings = pegagus_tokenizer(batch["dialogue"], max_length=1024, truncation=True)


  # Tokenizer knows it is tokenizing for decoder (encoder steps may be different)
  target_encodings = pegagus_tokenizer(text_target=batch["summary"], max_length=128, truncation=True)

  return {
      "input_ids": input_encodings["input_ids"],
      "attention_mask": input_encodings["attention_mask"],
      "labels": target_encodings["input_ids"]
  }


samsum_dataset = samsum_dataset.map(tokenize_dataset, batched=True)

columns = ["input_ids", "attention_mask", "labels"]

samsum_dataset.set_format(type="torch", columns=columns)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

# Create Data Collator

In [5]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=pegagus_tokenizer, model=pegasus_model)

# Set hyper-parameters

In [6]:
from transformers import Seq2SeqTrainingArguments

batch_size = 1 # Because model is big
logging_steps = 10

training_args = Seq2SeqTrainingArguments(output_dir="pegasus-samsum",
                                  num_train_epochs=1,
                                  warmup_steps=500,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  logging_steps=logging_steps,
                                  eval_strategy="steps",
                                  eval_steps=500,
                                  save_steps=1e6,

                                  # Needed when batch size is low to not hurt convergence
                                  gradient_accumulation_steps=16, # Make smaller batches & aggregate gradients (saves GPU memory)

                                  disable_tqdm=False,
                                  report_to="none", # Disable WandB logging

                                  # Evaluate generations as part of training loop
                                  predict_with_generate=True,
                                  generation_max_length=128
                                  )

# Train Model

In [ ]:
from transformers import Trainer

trainer = Trainer(model=pegasus_model,
                  args=training_args,
                  data_collator=data_collator,
                  train_dataset=samsum_dataset["train"],
                  eval_dataset=samsum_dataset["validation"])

trainer.train()